In [1]:
import sys
sys.path.append("/homes/gf332/compression-without-quantization/code")

import os, glob
from tqdm import tqdm as tqdm

import tensorflow.compat.v1 as tf
import tensorflow_compression as tfc
import tensorflow_probability as tfp
import tensorflow.contrib.eager as tfe
tfd = tfp.distributions
tfk = tf.keras
tfl = tf.keras.layers
tfq = tf.quantization

from binary_io import to_bit_string, from_bit_string

from pln import ProbabilisticLadderNetwork
from vae import VariationalAutoEncoder

from miracle import create_dataset, quantize_image, read_png

from coded_greedy_sampler import code_grouped_greedy_sample, decode_grouped_greedy_sample
from coded_importance_sampler import code_grouped_importance_sample, decode_grouped_importance_sample

import matplotlib.pyplot as plt
import numpy as np

tf.enable_eager_execution()

/scratch/gf332/compression_venv/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/scratch/gf332/compression_venv/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/scratch/gf332/compression_venv/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/scratch/gf332/compression_venv/lib/python3.5/site-pack

In [2]:
n_bits_per_step = 14
n_steps = 30
seed = 42
rho = 1.
first_level_max_group_size_bits=12
second_level_max_group_size_bits=4

with tf.Session() as sess:
    

#     res = code_greedy_sample_(t_loc=q1_loc,
#                               t_scale=q1_scale,
#                               p_loc=p1_loc,
#                               p_scale=p1_scale,
#                                 n_bits_per_step=14, 
#                                 n_steps=30, 
#                                 seed=seed, 
#                                 rho=1.)
    
#     bs, si = sess.run(res)
    
#     res = decode_greedy_sample_(sample_index=si, 
#                                   p_loc=p1_loc,
#                                   p_scale=p1_scale,
#                                   n_bits_per_step=14, 
#                                   n_steps=30, 
#                                   seed=seed, 
#                                   rho=1.)
#     samp = sess.run(res)

    res = code_grouped_greedy_sample_(sess=sess,
                                    target=q1, 
                                   proposal=p1,
                                   n_steps=n_steps, 
                                   n_bits_per_step=n_bits_per_step,
                                   seed=seed,
                                   max_group_size_bits=12,
                                   adaptive=True,
                                   backfitting_steps=0,
                                   use_log_prob=False,
                                   rho=1.)
    
    sample, bitcode, group_start_indices = res
    
    dec = decode_grouped_greedy_sample_(sess=sess,
                                  bitcode=bitcode, 
                                 group_start_indices=group_start_indices,
                                 proposal=p1, 
                                 n_bits_per_step=n_bits_per_step, 
                                 n_steps=n_steps, 
                                 seed=seed,
                                 adaptive=True,
                                 rho=1.)

NameError: name 'code_grouped_greedy_sample_' is not defined

In [7]:
x1 = b"01010"
x2 = b"101010"

x1.decode("utf-8") + x2.decode("utf-8")

'01010101010'

In [4]:
pln = ProbabilisticLadderNetwork(first_level_filters=196,
                 second_level_filters=128,
                 first_level_latent_channels=96,
                 second_level_latent_channels=24)

In [5]:
pln(tf.zeros((1, 256, 256, 3)))

<tf.Tensor: id=1579, shape=(1, 256, 256, 3), dtype=float32, numpy=
array([[[[-1.6792757e-02,  2.2184392e-03,  1.9746967e-02],
         [ 6.6196052e-03, -2.2168769e-02, -1.6886838e-02],
         [-2.1191034e-02, -2.8058609e-02,  7.4476851e-03],
         ...,
         [-1.0100249e-03,  3.9053417e-03, -7.7782874e-03],
         [ 5.2362448e-03,  5.4826005e-04, -5.9118317e-03],
         [ 2.3379992e-04,  3.7516505e-03,  2.7051612e-03]],

        [[-1.3277968e-02, -2.2607055e-02,  3.9487528e-03],
         [ 2.4542399e-03, -2.4774945e-03, -1.4310884e-03],
         [-2.9393932e-02,  1.3712058e-02, -2.8937060e-02],
         ...,
         [-3.3044368e-03,  8.9179445e-03, -6.7474688e-03],
         [ 7.3202178e-03, -5.6522200e-03,  2.7930634e-03],
         [ 9.4084884e-04,  3.5936534e-03,  3.3889655e-03]],

        [[ 4.1583637e-03, -1.6088195e-02, -3.2119947e-03],
         [-2.8121158e-02, -1.2608425e-02,  8.1854174e-04],
         [-1.9442908e-02, -7.2216660e-02, -8.9343907e-03],
         ...,
  

In [6]:
print(pln.posterior_1.loc.shape)
print(pln.posterior_2.loc.shape)
print(pln.prior_1.loc.shape)
print(pln.prior_2.loc.shape)

(1, 16, 16, 96)
(1, 4, 4, 24)
(1, 16, 16, 96)
(1, 4, 4, 24)
